In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

Прочитаем всё, less показал, что в каждой строчке метка отделена от сообщения табом.

In [5]:
X, y = [], []
with open("SMSSpamCollection.txt", "r") as f:
    for s in f.readlines():
        label, text = s.rstrip().split('\t')
        X.append(text)
        y.append(1 if label == 'spam' else 0)

print X[:5]
print y[:5]

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'Ok lar... Joking wif u oni...', "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", 'U dun say so early hor... U c already then say...', "Nah I don't think he goes to usf, he lives around here though"]
[0, 0, 1, 0, 0]


In [16]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
features = count_vectorizer.fit_transform(X)

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score

print features.shape, len(y)

lr = LogisticRegression()
cvs = cross_val_score(lr, features, y, cv=10, scoring='f1_macro')
print cvs
print np.average(cvs)

(5574, 8713) 5574
[ 0.98041322  0.9421179   0.95158631  0.97608025  0.9639066   0.95038677
  0.968107    0.95863465  0.95532864  0.9719125 ]
0.961847383452


Посмотрим, что мы думаем про эти сообщения:

In [24]:
X_test = """FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB
FreeMsg: Txt: claim your reward of 3 hours talk time
Have you visited the last lecture on physics?
Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$
Only 99$""".split('\n')
lr.fit(features, y)
print lr.predict(count_vectorizer.transform(X_test))

[1 1 0 0 0]


Удивительно, но 4 сообщение не спам как будто бы. Видимо, недообучили

In [30]:
def test_grams(*args, **kwargs):
    cv = CountVectorizer(ngram_range=args)
    Xs = cv.fit_transform(X)
    if kwargs.get('is_mult'):
        from sklearn.naive_bayes import MultinomialNB
        mnb = MultinomialNB()
        cvs = cross_val_score(mnb, Xs, y, cv=10, scoring='f1_macro')
        return np.average(cvs)
    else:
        lr = LogisticRegression()
        cvs = cross_val_score(lr, Xs, y, cv=10, scoring='f1_macro')
        return np.average(cvs)

In [61]:
print test_grams(2, 2)
print test_grams(3, 3)
print test_grams(1, 3)
print test_grams(2, 2, is_mult=True)
print test_grams(3, 3, is_mult=True)
print test_grams(1, 3, is_mult=True)
print test_grams(1, 1, is_mult=True)

0.899824452858
0.846370632349
0.957240374294
0.777510604526
0.521830718622
0.93427411872
0.957781061319


И правда, наивный Байес не особенно приветствует биграммы, а особенно триграммы

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
fs = tfidf.fit_transform(X)
lr = LogisticRegression()
cvs = cross_val_score(lr, fs, y, cv=10, scoring='f1_macro')
print np.average(cvs)

0.916619335048


**Выводы**: униграммы нужны, без них тяжело, логистическая регрессия лучше Байеса умеет выживать в условиях нехватки ресурсов.